In [62]:
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Input, Flatten, Dropout, Activation,  Dense, Embedding
from keras.callbacks import ModelCheckpoint
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import os
import yaml
from functools import partial
from sklearn.preprocessing import LabelEncoder
from ast import literal_eval
import re

# PATHS
base = ''
features_fp = os.path.join(base, 'training_X_power.npy')
labels_fp = os.path.join(base, 'sample_class.csv')
config_fp = os.path.join(base, 'config_power.yaml')

# DATA
features = np.load(features_fp)
labels = pd.read_csv(labels_fp, delimiter=",", index_col=0)

predicted_effects = ['overdrive.gain_db', 'reverb.reverberance', 'chorus.delays']

def find_on_effects(path):
    effects = yaml.load(open(path, 'r'))
    used = effects.get('DataAugmentation')
    active = used.get('active')
    sustain = used.get('sustain')
    r = used.get('effects')
    random_subeffects = [[k for k,v in r.get(a).items() if v.get('state') == 'random'] for a in active]
    return {a: random_subeffects[ix] for ix, a in enumerate(active)}, active, sustain

def one_hot_binarize(labels, predict_effects):
    
    abbrv = [x.split('.') for x in predict_effects]
    abbrv = [(x[0][:2] + '.' + x[-1][:2]).upper() for x in abbrv]
    binarize = labels[predict_effects].fillna(0)
    binarize = binarize.where(binarize == 0).replace(np.nan, 1)
    def labelize(row, abbrv):
        return ':__'.join(e + str(int(r)) for e,r in zip(abbrv, list(tuple(row))))
    labels = binarize.apply(lambda x: labelize(x, abbrv), axis = 1)
    encoder = LabelEncoder()
    one_hot = np_utils.to_categorical(encoder.fit_transform(labels))
    states = list(encoder.classes_)
    return one_hot, states

def list_check(labels, t = list):
    """Some parameters are given as lists"""
    labels_sub = labels.sample(n=10, random_state = 666).applymap(str)
    labels_sub = labels_sub.applymap(lambda x: True if re.search(r'\[', x) else False)
    labels_sub = labels_sub.any(axis = 0).loc[lambda x: x == True]
    return labels_sub.keys().tolist()

def format_data(config=config_fp, predicted_effects=['overdrive.gain_db', 
                                                         'reverb.reverberance', 
                                                         'chorus.delays'] ):
    
    
    random_fx, active, sustain = find_on_effects(config_fp)
    print ('Active effects: {}'.format(active))
    print ('Persisting {} during augmentation'.format(sustain))
    print ('Parameters allowed randomization {}'.format(random_fx))

    features = globals()['features']
    labels = globals()['labels'][predicted_effects]
    
    omit = list_check(labels)
    
    [*data] = train_test_split(features, labels, shuffle=True, test_size=0.2, random_state=44)
    shapes = list(map(lambda x: x.shape, data))    
    
    X_train, X_val, Y_train, Y_val = data
    
    # shared model, need binarized
    Y_train_bin, Y_train_states = one_hot_binarize(Y_train, predict_effects=predicted_effects)
    Y_val_bin, Y_val_states = one_hot_binarize(Y_val, predict_effects=predicted_effects)
    return (X_train, X_val, Y_train, Y_train_bin, Y_train_states, Y_val, Y_val_bin, Y_val_states, shapes)


# MAKE DATA

In [63]:
X_train, X_val, Y_train, \
Y_train_bin, Y_train_states, \
Y_val, Y_val_bin, Y_val_states, shapes = format_data()

Active effects: ['overdrive', 'reverb', 'chorus', 'phaser']
Persisting ['groko'] during augmentation
Parameters allowed randomization {'overdrive': ['gain_db'], 'reverb': ['reverberance'], 'chorus': [], 'phaser': []}


In [6]:
shapes

[(3888, 205, 1), (972, 205, 1), (3888, 3), (972, 3)]

In [8]:
# Both standard feature sets
Y_val.head(), Y_train.head()
Y_val.replace(np.nan, 0, inplace = True), Y_train.replace(np.nan, 0, inplace = True)

(None, None)

In [9]:
Y_train.iloc[:, :2]

,overdrive.gain_db,reverb.reverberance
498,66.0,0.0
4569,61.0,0.0
61,0.0,0.0
1852,0.0,0.0
4420,28.0,73.0
1586,46.0,86.0
1438,0.0,96.0
4292,0.0,75.0
1391,0.0,0.0
1040,0.0,90.0


In [12]:
from keras import backend as K

K.clear_session()

inputs = Input(shape = X_train.shape[1:])

shared = Conv1D(128, 5, padding='same', activation = 'relu')(inputs)
shared = Conv1D(128, 5, padding='same', activation = 'relu')(shared)
shared = Dropout(0,1)(shared)
shared =  MaxPooling1D(pool_size=8)(shared)
shared = Conv1D(128, 5, padding='same', activation = 'relu')(shared)
shared = Conv1D(128, 5, padding='same', activation = 'relu')(shared)
shared = Conv1D(128, 5, padding='same', activation = 'relu')(shared)
shared = Dropout(0.3)(shared)
shared = Conv1D(128, 5, padding='same', activation = 'relu')(shared)
shared = Flatten()(shared)

classifier = Dense(len(globals()['Y_train_states']), activation='softmax', name='classifier')(shared)
regressor = Dense(1, activation='relu', name='regressor')(shared)

model = Model(inputs=inputs, outputs=[classifier, regressor]) # regressor
optimizer = keras.optimizers.Adam(lr=0.00001)

model.compile(
            optimizer=optimizer,
            loss = {
                'regressor' : 'mae',
                'classifier': 'categorical_crossentropy'},
            metrics = {'classifier' : 'accuracy'}
)

model.fit(X_train, [Y_train_bin, Y_train.iloc[:, :1]],
        batch_size=32, 
        epochs=30), 
        validation_data=(X_val, Y_val),



Epoch 1/30
3888/3888 [==============================] - 19s 5ms/step - loss: 25.0220 - classifier_loss: 1.9757 - regressor_loss: 23.0463 - classifier_acc: 0.1525
Epoch 2/30
3888/3888 [==============================] - 17s 4ms/step - loss: 24.0908 - classifier_loss: 1.7863 - regressor_loss: 22.3045 - classifier_acc: 0.2657
Epoch 3/30
3888/3888 [==============================] - 17s 4ms/step - loss: 16.8839 - classifier_loss: 1.6127 - regressor_loss: 15.2711 - classifier_acc: 0.3236
Epoch 4/30
3888/3888 [==============================] - 18s 5ms/step - loss: 12.8841 - classifier_loss: 1.5275 - regressor_loss: 11.3566 - classifier_acc: 0.3578
Epoch 5/30
 992/3888 [======>.......................] - ETA: 14s - loss: 11.3311 - classifier_loss: 1.4820 - regressor_loss: 9.8491 - classifier_acc: 0.3972

KeyboardInterrupt: 